In [2]:
#https://github.com/aligator4sah/Kaggle_Data_Analytics/blob/master/data/test.csv

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
#%matplotlib inline 
from sklearn.preprocessing import LabelEncoder 

 
train_data = pd.read_csv("C:/Study/kaggle_Pro/train.csv") 
test_data = pd.read_csv("C:/Study/kaggle_Pro/test.csv")  
 
#print("Train data dimensions: ", train_data.shape) 
#print("Test data dimensions: ", test_data.shape) 
train_data = train_data.iloc[:,1:]  
 
train_data.info() 
test_data.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 131 entries, cat1 to loss
dtypes: float64(15), object(116)
memory usage: 188.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125546 entries, 0 to 125545
Columns: 131 entries, id to cont14
dtypes: float64(14), int64(1), object(116)
memory usage: 125.5+ MB


In [3]:
#Updating loss 
train_data['loss'] = np.log1p(train_data["loss"])  
 
# sepearte the categorical and continous features 
cont_columns = [] 
cat_columns = []  
 
for i in train_data.columns: 
    if train_data[i].dtype == 'float': 
        cont_columns.append(i) 
    elif train_data[i].dtype == 'object': 
        cat_columns.append(i) 
 
for cf1 in cat_columns: 
    le = LabelEncoder() 
    le.fit(train_data[cf1].unique()) 
    train_data[cf1] = le.transform(train_data[cf1]) 
 
#train_data.head(20) 


In [4]:
ntrain = train_data.shape[0] 
ntest = test_data.shape[0]  
 
features = [x for x in train_data.columns if x not in ['id','loss']]  
 
train_test = pd.concat((train_data[features], test_data[features])).reset_index(drop=True) 
 
#train_test = np.concatenate((train_data_pred[features],test_data[features]),axis=0) 
 
for f in train_test.columns:  
    if train_test[f].dtype=='object':  
        lbl = LabelEncoder()  
        lbl.fit(list(train_test[f].values))  
        train_test[f] = lbl.transform(list(train_test[f].values)) 
 
train_x = train_test.iloc[:ntrain,:] 
 
test_x = train_test.iloc[ntrain:,:] 
    
train_x = np.array(train_x); 
test_x = np.array(test_x); 
#print(test_x) 


In [5]:
#get the number of rows and columns 
r, c = train_x.shape 
 
#Y is the target column, X has the rest 
X = train_x 
Y = train_data['loss'] 
 
#print X.shape 
print ntrain 
 
#Validation chunk size 
val_size = 0.4 
 
#Use a common seed in all experiments so that same chunk is used for validation 
seed = 0  
 
from sklearn import cross_validation 
X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(X, Y, test_size=val_size, random_state=seed) 
 
del X 
del Y 
 
i_cols = [] 
for i in range(0,c-1): 
    i_cols.append(i) 
 
#All features 
X_all = [] 
 
#List of combinations 
comb = [] 
 
#Dictionary to store the MAE for all algorithms  
mae = [] 
 
#Scoring parameter 
from sklearn.metrics import mean_absolute_error 
 
#Add this version of X to the list  
n = "All" 
#X_all.append([n, X_train,X_val,i_cols]) 
X_all.append([n, i_cols]) 


188318


C:\Users\daisy\AppData\Roaming\Python\Python27\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
#Evaluation of various combinations of LinearRegression 
 
#Import the library 
from sklearn.linear_model import LinearRegression 
import numpy as numpy
 
##Set the base model 
model = LinearRegression(n_jobs=-1) 
algo = "LR" 
# 
##Accuracy of the model using all features 
for name,i_cols_list in X_all: 
    model.fit(X_train[:,i_cols_list],Y_train) 
    result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list]))) 
    mae.append(result) 
    print(name + " %s" % result) 
comb.append(algo) 


All 1295.70094426


In [11]:
#Evaluation of various combinations of ElasticNet LinearRegression

from sklearn.linear_model import ElasticNet

#When alpha equals to 0.001,I get the lowest MAE.
a_list = numpy.array([0.001])

for alpha in a_list:
    #Set the base model
    model = ElasticNet(alpha=alpha,random_state=seed)
    
    algo = "Elastic"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % alpha )

#all the non-liner regression model get worse result than liner regression, so I didn't try here.

All 1295.06901455


In [9]:
#Evaluation of various combinations of RandomForest
import numpy as numpy
#Import the library
from sklearn.ensemble import RandomForestRegressor

#When the n_estimator equals to 50, we can get the lowest MAE.
n_list = numpy.array([50])

for n_estimators in n_list:
    #Set the base model
    model = RandomForestRegressor(n_jobs=-1,n_estimators=n_estimators,random_state=seed)
    
    algo = "RF"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_estimators )

#will try the extremly randomized tree later

All 1232.13382429


In [20]:
#Evaluation of various combinations of Bagged Decision Trees

#Import the library
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

n_list = numpy.array([100])

for n_estimators in n_list:
    #Set the base model
    model = BaggingRegressor(n_jobs=-1,n_estimators=n_estimators)
    
    algo = "Bag"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_estimators )


All 1226.19538847


In [18]:
#Evaluation of various combinations of AdaBoost

#Import the library
from sklearn.ensemble import AdaBoostRegressor

#When the n_estimator equals to 100. I get the lowest MAE.
n_list = numpy.array([100])

for n_estimators in n_list:
    #Set the base model
    model = AdaBoostRegressor(n_estimators=n_estimators,random_state=seed)
    
    algo = "Ada"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_estimators )

#The Mae is too high so I don't think Adaboost is fitful for this dataset.

All 1608.15856307


In [17]:
#Evaluation of various combinations of SGB (Sophisticated Gradient Boosting)

#Import the library
from sklearn.ensemble import GradientBoostingRegressor

#I tried n_estimator from 50-250 and I found the higher n_estimator, the lower MAE I got.
n_list = numpy.array([250])

for n_estimators in n_list:
    #Set the base model
    model = GradientBoostingRegressor(n_estimators=n_estimators,random_state=seed)
    
    algo = "SGB"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_estimators )


All 1185.27819913


In [9]:
#Evaluation of various combinations of KNN

#Import the library
import numpy
from sklearn.neighbors import KNeighborsRegressor

#Add the N value to the below list if you want to run the algo
n_list = numpy.array([10])

for n_neighbors in n_list:
    #Set the base model
    model = KNeighborsRegressor(n_neighbors=n_neighbors,n_jobs=-1)
    
    algo = "KNN"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_neighbors )

All 1702.61267621


In [ ]:
#All the preperations I used the same way as Liju did.

#I also tried MLP on my laptop, but it keeps running for an hour and nothing happened. 
#I'm wondering if there is any dead loop inside the MLP.
#I think we can check it next time when we meet.

#In conclusion, SGB get the best result and Adaboost get the worst result.
#For regression and bagging, their result are just so so but I think we can make a change by diff ways of combination.
#Most of the model above, I used all features to fit them.
#I think we should find some ways to identify which model fit for which features and combine them.
#Second thing is that we should know how to tune the parameter for each model. 
#Once we decide what models we are going to use, we can divide the model to each of us and combine them at last.